In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import random
from random import choices
from scipy import stats
import pickle
from scipy.stats import chi2_contingency
import imblearn
from imblearn.under_sampling import TomekLinks
from ctgan import CTGAN
import torch
import sdmetrics
from sdmetrics.reports.single_table import QualityReport
from sdmetrics.single_column import TVComplement, KSComplement

# Load data

In [ ]:
testlabels = pd.read_csv("data/ltc_events_readmission_merged_admimeth_inputs_final_test.tsv",sep='\t')
testinput = pd.read_csv("data/ltc_events_readmission_merged_admimeth_labels_final_test_nopostadmission.tsv",sep='\t')
print(testinput.shape, testlabels.shape)

(13906, 21) (309232, 204)


In [ ]:
testinput.head(3)

,eid,first_admi_date,first_discharge_date,second_admi_date,first_admi_duration,diag,admi_meth,delta_length,readmission_state,sex,...,ltc_count,townsend_deprivation_index,alcohol_intake_frequency,smoking_status,relative_smoking_pack_years,ethnic_background,BMI,body_fat_percentage,historical_scripts,preadmission_scripts
0,1000620,2016-08-08,2016-08-08,2016-08-28,17,C160,2000,3,True,M,...,12,-2.39987,4.0,1.0,0.2625,1001.0,33.8889,30.0,"0103050,0205052,0209000,0212000,0406000,040701...",none
1,1000833,2000-03-16,2000-03-16,2008-01-29,0,K625,1002,2875,False,F,...,1,-2.18021,5.0,0.0,0.0000,1001.0,23.5284,28.1,"0107020,0407010,1001010",none
2,1000857,2019-11-21,2019-11-21,2019-11-22,0,C509,2001,1,True,F,...,4,-3.88234,6.0,0.0,0.0000,1001.0,40.9347,48.2,none,none


In [ ]:
testlabels.tail(3)

,patient_id,event_date,delta_time,PMR,glaucoma,OA,erectile_dysfunction,pri_skin,spondylosis,obesity,...,cerebral_palsy,pri_mesothelioma,pri_brain,thala,collapsed_vert,sick_sinus,ADHD,juv_arth,entero_arthro,SARS_CoV2
309229,6024693,2015-08-03,77,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
309230,6024693,2015-09-15,43,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
309231,6024693,2015-10-12,27,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


# DATASET CURATION

## Fix LTCs

In [ ]:
# Modify to 1 all floats bigger than 0.
testlabels.iloc[:,3:] = np.ceil(testlabels.iloc[:,3:].values)

## Binarize prescriptions

In [ ]:
# Historical
histpresc = testinput['historical_scripts'].str.cat(sep=',')
histpresc_list = re.split(',\s*', histpresc)
unique_histpresc = list(set(histpresc_list))

for word in unique_histpresc:
    name = 'hist_' + word
    testinput[name] = testinput['historical_scripts'].str.contains(
        word).astype(int)

# Preadmissiontestinput
pread = testinput['preadmission_scripts'].str.cat(sep=',')
pread_list = re.split(',\s*', pread)
unique_pread = list(set(pread_list))

for word in unique_pread:
    name = 'pread_' + word
    testinput[name] = testinput['preadmission_scripts'].str.contains(
        word).astype(int)

testinput.drop('preadmission_scripts', axis = 1, inplace = True)
testinput.drop('historical_scripts', axis = 1, inplace = True)

C:\Users\hugoc\AppData\Local\Temp\ipykernel_4508\2387465891.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  testinput[name] = testinput['historical_scripts'].str.contains(
C:\Users\hugoc\AppData\Local\Temp\ipykernel_4508\2387465891.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  testinput[name] = testinput['historical_scripts'].str.contains(
C:\Users\hugoc\AppData\Local\Temp\ipykernel_4508\2387465891.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times

C:\Users\hugoc\AppData\Local\Temp\ipykernel_4508\2387465891.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  testinput[name] = testinput['historical_scripts'].str.contains(
C:\Users\hugoc\AppData\Local\Temp\ipykernel_4508\2387465891.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  testinput[name] = testinput['historical_scripts'].str.contains(
C:\Users\hugoc\AppData\Local\Temp\ipykernel_4508\2387465891.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times

C:\Users\hugoc\AppData\Local\Temp\ipykernel_4508\2387465891.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  testinput[name] = testinput['historical_scripts'].str.contains(
C:\Users\hugoc\AppData\Local\Temp\ipykernel_4508\2387465891.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  testinput[name] = testinput['historical_scripts'].str.contains(
C:\Users\hugoc\AppData\Local\Temp\ipykernel_4508\2387465891.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times

C:\Users\hugoc\AppData\Local\Temp\ipykernel_4508\2387465891.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  testinput[name] = testinput['historical_scripts'].str.contains(
C:\Users\hugoc\AppData\Local\Temp\ipykernel_4508\2387465891.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  testinput[name] = testinput['historical_scripts'].str.contains(
C:\Users\hugoc\AppData\Local\Temp\ipykernel_4508\2387465891.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times

C:\Users\hugoc\AppData\Local\Temp\ipykernel_4508\2387465891.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  testinput[name] = testinput['historical_scripts'].str.contains(
C:\Users\hugoc\AppData\Local\Temp\ipykernel_4508\2387465891.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  testinput[name] = testinput['historical_scripts'].str.contains(
C:\Users\hugoc\AppData\Local\Temp\ipykernel_4508\2387465891.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times

...

C:\Users\hugoc\AppData\Local\Temp\ipykernel_4508\2387465891.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  testinput[name] = testinput['preadmission_scripts'].str.contains(
C:\Users\hugoc\AppData\Local\Temp\ipykernel_4508\2387465891.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  testinput[name] = testinput['preadmission_scripts'].str.contains(
C:\Users\hugoc\AppData\Local\Temp\ipykernel_4508\2387465891.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` man

C:\Users\hugoc\AppData\Local\Temp\ipykernel_4508\2387465891.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  testinput[name] = testinput['preadmission_scripts'].str.contains(
C:\Users\hugoc\AppData\Local\Temp\ipykernel_4508\2387465891.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  testinput[name] = testinput['preadmission_scripts'].str.contains(
C:\Users\hugoc\AppData\Local\Temp\ipykernel_4508\2387465891.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` man

C:\Users\hugoc\AppData\Local\Temp\ipykernel_4508\2387465891.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  testinput[name] = testinput['preadmission_scripts'].str.contains(
C:\Users\hugoc\AppData\Local\Temp\ipykernel_4508\2387465891.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  testinput[name] = testinput['preadmission_scripts'].str.contains(
C:\Users\hugoc\AppData\Local\Temp\ipykernel_4508\2387465891.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` man

C:\Users\hugoc\AppData\Local\Temp\ipykernel_4508\2387465891.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  testinput[name] = testinput['preadmission_scripts'].str.contains(
C:\Users\hugoc\AppData\Local\Temp\ipykernel_4508\2387465891.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  testinput[name] = testinput['preadmission_scripts'].str.contains(
C:\Users\hugoc\AppData\Local\Temp\ipykernel_4508\2387465891.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` man

C:\Users\hugoc\AppData\Local\Temp\ipykernel_4508\2387465891.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  testinput[name] = testinput['preadmission_scripts'].str.contains(
C:\Users\hugoc\AppData\Local\Temp\ipykernel_4508\2387465891.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  testinput[name] = testinput['preadmission_scripts'].str.contains(
C:\Users\hugoc\AppData\Local\Temp\ipykernel_4508\2387465891.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` man

## Merge binarized prescriptions & LTC BOWs

In [ ]:
sorted_ltcs = testlabels.sort_values('event_date', ascending=False).groupby('patient_id').head(1)
ltc_columns = sorted_ltcs.iloc[:, 3:]
prefixed_ltcs = ltc_columns.add_prefix('ltc_')

mergeddf = testinput.join(prefixed_ltcs, on='eid')

## Binarize diags

In [ ]:
diag_dict = pickle.load( open("top_250_merged_diags.pkl", "rb") )

one_hot_encoded = pd.get_dummies(mergeddf['diag'])

# Only top 250 diags
keys = diag_dict.keys()
filtered_df = one_hot_encoded[[col for col in one_hot_encoded.columns if col in keys]]
prefixed_df = filtered_df.add_prefix('diag_')

mergeddf = pd.concat([mergeddf, prefixed_df], axis=1)
mergeddf.drop('diag', axis = 1, inplace = True)

## Define variable metada

In [ ]:
admission = list(mergeddf.columns)[:7]

target = list(mergeddf.columns)[7]

demographics = list(mergeddf.columns)[8:18]
hist_drugs =  list(mergeddf.columns)[18:1054-540]
pread_drugs = list(mergeddf.columns)[1054-540:1054-202]

ltcs = list(mergeddf.columns)[1054-202:1053]
diags = list(mergeddf.columns)[1053:]

In [ ]:
# list
print(admission[0],admission[-1])
print(target)
print(demographics[0],demographics[-1])
print(hist_drugs[0],hist_drugs[-1])
print(pread_drugs[0],pread_drugs[-1])
print(ltcs[0],ltcs[-1])
print(diags[0],diags[-1])

eid delta_length
readmission_state
sex body_fat_percentage
hist_1314000 pread_1103010
pread_0301050 ltc_crohns
ltc_thrombophilia diag_N993
diag_O001 diag_Z466


In [ ]:
binary = ['sex'] + hist_drugs + pread_drugs + ltcs + diags
target = target
continuous = ['age_at_event', 'ltc_count', 'townsend_deprivation_index', 'alcohol_intake_frequency', 'smoking_status',
              'relative_smoking_pack_years', 'ethnic_background', 'BMI', 'body_fat_percentage', 'first_admi_duration']
toremove = ['eid', 'first_admi_date','first_discharge_date', 'second_admi_date', 'delta_length', 'admi_meth']

## Deal missing values

In [ ]:
print('Number of NaNs:', mergeddf.isna().sum().sum())

Number of NaNs: 2795106


In [ ]:
columns_with_nan = mergeddf.columns[mergeddf.isna().any()].tolist()
mergeddf[columns_with_nan] = mergeddf[columns_with_nan].fillna(0)

## Transform gender column

In [ ]:
# Male = 0, Female = 1
mapping = {'M': 0, 'F': 1}
mergeddf['sex'] = mergeddf['sex'].map(mapping)

In [ ]:
mergeddf.to_excel('data/test_data_fixedltcs.xlsx',index = False )